# MDL DB read

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['lines.linewidth'] = 1.5

import os
import copy

import data
from mdldb.mdl_db import MDLDataBase
from mdldb.tables import Run
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.analytical_linear_estimator import AnalyticalLinearEstimator
from rolldecayestimators.direct_linear_estimator import DirectLinearEstimator
from rolldecayestimators.direct_estimator_cubic import DirectEstimatorCubic
from rolldecayestimators.direct_estimator import DirectEstimator
from rolldecayestimators.norwegian_estimator import NorwegianEstimator



from mdldb import mdl_to_evaluation
from evaluation.run_dynamic import RunDynamic
from evaluation.run_manoeuvring import RunZigZag
from sklearn.pipeline import Pipeline
import signal_lab

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///' + data.mdl_db_path)
db = MDLDataBase(engine=engine)

In [ ]:
db.session.query(Run.test_type).distinct().all()

In [ ]:
len(db.session.query(Run).filter(Run.test_type=='roll decay').all())

In [ ]:
roll_decay_tests = db.session.query(Run).filter(Run.test_type=='roll decay')

In [ ]:
db_run = roll_decay_tests[100]
#db_run = roll_decay_tests[150]

In [ ]:
ascii_file = db_run.load()
df_raw = ascii_file.channels

df = signal_lab.mdl_to_evaluation.do_transforms(df=df_raw)
df.rename(columns={'MA/Roll':'phi'}, inplace=True)

In [ ]:
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=2, minimum_score=0.99)
scaler = ScaleFactorTransformer(scale_factor=db_run.model.scale_factor)  # dummy value None for now
cutter = CutTransformer(phi_max=np.deg2rad(7), phi_min=np.deg2rad(1))

steps = [('filter',lowpass_filter),
         ('scaler',scaler),
         ('cutter', cutter),
        ]

preprocess = Pipeline(steps)
X = preprocess.fit_transform(df)

In [ ]:
X.plot(y='phi')

In [ ]:
fig,ax=plt.subplots()
df.plot(y='Carriage/Psip',ax=ax)

In [ ]:
X.head()

In [ ]:
t=X.index
phi=X['phi']
N=5
t2=np.linspace(t[0],t[-1],N*len(t))

X2 = pd.DataFrame(index=t2)
keys=['phi','phi1d','phi2d','phi_filtered']
for key in keys:
    X2[key]=np.interp(t2,t,X[key])

In [ ]:
fig,ax=plt.subplots()
X2.plot(y='phi', ax=ax)

In [ ]:
estimator=DirectEstimator(omega_regression=True, fit_method='integration')
df_temp = estimator.simulate(t=X.index,phi0=X.iloc[0]['phi'],phi1d0=X.iloc[0]['phi1d'],omega0=0.43,zeta=0.1,d=0.1)
df_temp.plot(y='phi')

In [ ]:
estimators = []
#estimators.append(DirectLinearEstimator(omega_regression=True))
#estimators.append(AnalyticalLinearEstimator(omega_regression=True))
estimators.append(DirectEstimator(omega_regression=True, fit_method='derivation'))
#estimators.append(NorwegianEstimator())
#estimators.append(DirectEstimatorCubic(omega_regression=True))

#estimators.append(DirectLinearEstimator(omega_regression=False))
#estimators.append(AnalyticalLinearEstimator(omega_regression=False))
estimators.append(DirectEstimator(omega_regression=False, fit_method='derivation'))
#estimators.append(NorwegianEstimator())
#estimators.append(DirectEstimatorCubic(omega_regression=False))

#estimators.append(DirectLinearEstimator(omega_regression=False))
#estimators.append(AnalyticalLinearEstimator(omega_regression=False))
estimators.append(DirectEstimator(omega_regression=True, fit_method='integration'))
#estimators.append(NorwegianEstimator())
#estimators.append(DirectEstimatorCubic(omega_regression=False))

#estimators.append(DirectLinearEstimator(omega_regression=False))
#estimators.append(AnalyticalLinearEstimator(omega_regression=False))
estimators.append(DirectEstimator(omega_regression=False, fit_method='integration'))
#estimators.append(NorwegianEstimator())
#estimators.append(DirectEstimatorCubic(omega_regression=False))


for estimator in estimators:

    estimator.fit(X)
    
    fig,ax=plt.subplots()
    fig.set_size_inches(14,10)
    estimator.plot_fit(ax=ax)
    score = estimator.score()
    
    title = ''
    if estimator.omega_regression:
        title+='Omega regression '
    else:
        title+='Omega fft '
    
    title+='%s ' % estimator.fit_method
    
    
    title+='Score:%0.2f' % score 
    ax.set_title(title)

In [ ]:
from scipy.optimize import curve_fit
import numpy as np

In [ ]:
class A():
    
    @staticmethod
    def f(x,f2,xs,ys):
        return ys-f2(x,xs)
    
    @staticmethod
    def f2(x,xs):
        return x[0] + x[1]*xs + x[1]*xs

In [ ]:
xs = np.linspace(0,10,100)
k = 1
m = 0
ys = k*xs+m

In [ ]:
a = A()
a.f(x=[m,k],f2=a.f2, xs=xs,ys=ys)

In [ ]:
from scipy.optimize import least_squares
import inspect

In [ ]:
result = least_squares(fun=a.f,x0=[1,1],kwargs={'f2':a.f2,'xs':xs,'ys':ys})

In [ ]:
result.x

In [ ]:
for i in range(1000):
    result2 = least_squares(fun=a.f,x0=[1,1],kwargs={'f2':a.f2,'xs':xs,'ys':ys})
    assert(result.x[1]==result2.x[1])

In [ ]:
df.plot(y='phi')

In [ ]:
df.plot(y='phi')

In [ ]:
DirectEstimator.calculate_acceleration

In [ ]:
class RollDecay():
    
    functions = (DirectEstimator.acceleration_lambda,)
    
    @property
    def calculate_acceleration(self):
        return self.functions[0]
    
    @property
    def parameter_names(self):
        signature = inspect.signature(self.calculate_acceleration)
        return list(set(signature.parameters.keys()) - set(['phi','phi1d']))
    
    @staticmethod
    def error(x,self,xs,ys):
        return ys-self.equation(x,xs)
    
    def equation(self,x,xs):
        
        parameters={key:x for key,x in zip(self.parameter_names,x)}
        
        phi=xs['phi']
        phi1d=xs['phi1d']
        
        acceleration = self.calculate_acceleration(phi=phi,phi1d=phi1d,**parameters)
        return acceleration
    
    def fit(self,X):
        
        kwargs={'self':self,
        'xs':X,
        'ys':X['phi2d']}

        self.result = least_squares(fun=self.error,x0=[0.5,0.5,0.5],kwargs=kwargs)
        self.parameters={key:x for key,x in zip(self.parameter_names,self.result.x)}
    
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from numpy import sqrt, pi, sin, cos
from scipy.integrate import ode

# use z = [z1, z2] = [u, u']
# and then f = z' = [u', u''] = [z2, -z1+sqrt(z1)]

class A:
    
    def f(self,phi, z):
        return [z[1], -z[0]+sqrt(z[0])]

a = A()
# initialize the 4th order Runge-Kutta solver
solver = ode(a.f).set_integrator('dopri5')

# initial value
z0 = [1.49907, 0.]
solver.set_initial_value(z0)

values = 10000
phi = np.linspace(0.0001, 7.*pi, values)
u = np.zeros(values)
#
for ii in range(values):
    u[ii] = solver.integrate(phi[ii])[0] #z[0]=u

x = 1. / u * cos(phi)
y = 1. / u * sin(phi)

plt.figure()
plt.plot(x,y)
plt.grid()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from numpy import sqrt, pi, sin, cos
from scipy.integrate import solve_ivp

# use z = [z1, z2] = [u, u']
# and then f = z' = [u', u''] = [z2, -z1+sqrt(z1)]

class A:
    
    def f(self,phi, z):
        return [z[1], -z[0]+sqrt(z[0])]

a = A()
# initialize the 4th order Runge-Kutta solver
t = phi
t_span = [np.min(t),np.max(t)]
result = solve_ivp(fun=a.f, t_span=t_span, y0=z0, t_eval=phi)
u = result.y[0,:]

#solver = ode(a.f).set_integrator('dopri5')
#
## initial value
#z0 = [1.49907, 0.]
#solver.set_initial_value(z0)
#
#values = 10000
#phi = np.linspace(0.0001, 7.*pi, values)
#u = np.zeros(values)
##
#for ii in range(values):
#    u[ii] = solver.integrate(phi[ii])[0] #z[0]=u
#
x = 1. / u * cos(phi)
y = 1. / u * sin(phi)

plt.figure()
plt.plot(x,y)
plt.grid()
plt.show()

In [ ]:
result